In [1]:
import pandas as pd
from datetime import datetime
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
def side_by_side(*objs, **kwds):
    from pandas.io.formats.printing import adjoin
    space = kwds.get('space', 4)
    reprs = [repr(obj).split('\n') for obj in objs]
    print (adjoin(space, *reprs))
    print()
    return

## Load Data 

In [2]:
admissions = pd.read_csv("admissions_3K.csv")

In [3]:
admissions.shape 

(3299, 22)

In [6]:
admissions.columns

Index(['HADM_ID', 'HeartRate', 'RespRate', 'SaO2', 'hrAlarmHigh', 'hrAlarmLow',
       'sao2AlarmHigh', 'sao2AlarmLow', 'temperature', 'SkinTemperature',
       'BPCuffDiastolic', 'BPCuffMean', 'BPCuffSystolic', 'Glucometer',
       'BirthWeight', 'HeadCirc', 'Bands', 'Eosinophils', 'Lymphs', 'Monos',
       'Neuts', 'Platelet'],
      dtype='object')

In [4]:
master_neonates = admissions.loc[(admissions.HeartRate==True) & (admissions.RespRate==True) & (admissions.SaO2==True) 
               & (admissions.hrAlarmHigh==True) & (admissions.hrAlarmLow==True) & (admissions.sao2AlarmHigh==True)
               & (admissions.sao2AlarmLow==True) & (admissions.BPCuffDiastolic==True) & (admissions.BPCuffMean==True)
               & (admissions.BPCuffSystolic==True) & (admissions.SkinTemperature==True) & (admissions.temperature==True)
               & (admissions.HeadCirc==True) & (admissions.Glucometer==True)
               & (admissions.Eosinophils==True)
               & (admissions.Lymphs==True) & (admissions.Monos==True) & (admissions.Neuts==True)
               & (admissions.Bands==True) & (admissions.Platelet==True)
               & (admissions.BirthWeight==True)
               ] 

In [5]:
master_neonates.shape 

(3145, 22)

In [39]:
master_neonates = master_neonates['HADM_ID'].to_frame()

## Defining Extracting Function 

In [81]:
def extract_24(df, master_ids, filename):
    ## Renaming columns 
    df.rename(columns = {'subject_id_x': 'subject_id'}, inplace=True) 
    
    ## Dropping extra columns 
    df.drop(['subject_id_y', 'DISCHTIME', 'Dead'], axis=1, inplace=True)
    
    ## Merging with master neonates 
    df = pd.merge(df, master_ids, on='HADM_ID', how='inner')
    
    ## Converting dates 
    df['CHARTTIME'] = df['CHARTTIME'].astype('datetime64[ns]')
    df['ADMITTIME'] = df['ADMITTIME'].astype('datetime64[ns]') 
    
    ## Calculating time interval 
    df['TIME_INTERVAL'] = df['CHARTTIME'] - df['ADMITTIME']

    ## Extracting only 24 hours data 
    df_24 = df[(df.TIME_INTERVAL <= '1 days') & (df.TIME_INTERVAL >= '-1 days')]

    df_24.drop(['TIME_INTERVAL'], axis=1, inplace=True) 
    
    ## Saving into a given file 
    df_24.to_csv(filename, index=False)
    

## 1 Load Chart Events 

## 1.1 Heart Rate 

In [86]:
heart_rate_df = pd.read_csv("Data_3173_TS/heart_rate.csv")  

In [40]:
heart_rate_df.head()

,subject_id_x,HADM_ID,CHARTTIME,VALUENUM,subject_id_y,ADMITTIME,DISCHTIME,Dead
0,10824,143406,2126-07-01 06:00:00,145.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0
1,10824,143406,2126-07-01 07:00:00,136.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0
2,10824,143406,2126-07-03 16:00:00,147.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0
3,10824,143406,2126-06-18 16:00:00,153.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0
4,10824,143406,2126-06-18 17:00:00,151.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0


In [41]:
heart_rate_df.rename(columns = {'subject_id_x': 'subject_id'}, inplace=True) 
heart_rate_df.drop(['subject_id_y', 'DISCHTIME', 'Dead'], axis=1, inplace=True)

In [42]:
heart_rate_df.head()

,subject_id,HADM_ID,CHARTTIME,VALUENUM,ADMITTIME
0,10824,143406,2126-07-01 06:00:00,145.0,2126-06-12 22:08:00
1,10824,143406,2126-07-01 07:00:00,136.0,2126-06-12 22:08:00
2,10824,143406,2126-07-03 16:00:00,147.0,2126-06-12 22:08:00
3,10824,143406,2126-06-18 16:00:00,153.0,2126-06-12 22:08:00
4,10824,143406,2126-06-18 17:00:00,151.0,2126-06-12 22:08:00


In [43]:
heart_rate_df['HADM_ID'].nunique()

3299

In [44]:
heart_rate_df.shape 

(1542782, 5)

In [45]:
heart_rate_df = pd.merge(heart_rate_df, master_neonates, on='HADM_ID', how='inner')

In [46]:
heart_rate_df['HADM_ID'].nunique()

3145

In [47]:
heart_rate_df.shape 

(1453910, 5)

In [48]:
heart_rate_df.head(2)

,subject_id,HADM_ID,CHARTTIME,VALUENUM,ADMITTIME
0,10824,143406,2126-07-01 06:00:00,145.0,2126-06-12 22:08:00
1,10824,143406,2126-07-01 07:00:00,136.0,2126-06-12 22:08:00


In [49]:
start = datetime.now()
heart_rate_df['CHARTTIME'] = heart_rate_df['CHARTTIME'].astype('datetime64[ns]')
heart_rate_df['ADMITTIME'] = heart_rate_df['ADMITTIME'].astype('datetime64[ns]')

print("Computing time takes %s" %(datetime.now()-start))

Computing time takes 0:00:01.028336


In [50]:
heart_rate_df['TIME_INTERVAL'] = heart_rate_df['CHARTTIME'] - heart_rate_df['ADMITTIME']

In [51]:
heart_rate_24 = heart_rate_df[(heart_rate_df.TIME_INTERVAL <= '1 days') & (heart_rate_df.TIME_INTERVAL >= '-1 days')]

In [52]:
heart_rate_24.shape 

(78178, 6)

In [53]:
heart_rate_24['HADM_ID'].nunique()

3145

In [54]:
heart_rate_24.to_csv('Data_3145_TS_24/heart_rate_24.csv', index=False)

In [87]:
extract_24(heart_rate_df, master_neonates, 'Data_3145_TS_24/heart_rate_24.csv')

## 1.2 Resp Rate 

In [84]:
resp_rate_df = pd.read_csv("Data_3173_TS/resp_rate.csv") 
resp_rate_df.head(2)

,subject_id_x,HADM_ID,CHARTTIME,VALUENUM,subject_id_y,ADMITTIME,DISCHTIME,Dead
0,1291,141087,2100-07-04 09:30:00,60.0,1291,2100-07-04 10:18:00,2100-07-23 15:30:00,0
1,1291,141087,2100-07-04 10:00:00,49.0,1291,2100-07-04 10:18:00,2100-07-23 15:30:00,0


In [85]:
extract_24(resp_rate_df, master_neonates, 'Data_3145_TS_24/resp_rate_24.csv')

## 1.3 SaO2 

In [82]:
sao2_df = pd.read_csv("Data_3173_TS/sao2.csv")  
sao2_df.head(2)

,subject_id_x,HADM_ID,CHARTTIME,VALUENUM,subject_id_y,ADMITTIME,DISCHTIME,Dead
0,10824,143406,2126-07-01 06:00:00,95.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0
1,10824,143406,2126-07-01 07:00:00,97.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0


In [83]:
extract_24(sao2_df, master_neonates, 'Data_3145_TS_24/sao2_24.csv')

## 1.4 HR Alarm High 

In [88]:
hr_alarm_high_df = pd.read_csv("Data_3173_TS/hr_alarm_high.csv")  
hr_alarm_high_df.head(2)

,subject_id_x,HADM_ID,CHARTTIME,VALUENUM,subject_id_y,ADMITTIME,DISCHTIME,Dead
0,69,190201,2129-03-25 03:00:00,200.0,69,2129-03-21 15:33:00,2129-04-09 13:30:00,0
1,69,190201,2129-03-25 04:00:00,200.0,69,2129-03-21 15:33:00,2129-04-09 13:30:00,0


In [89]:
extract_24(hr_alarm_high_df, master_neonates, 'Data_3145_TS_24/hr_alarm_high_24.csv')

## 1.5 HR Alarm Low 

In [90]:
hr_alarm_low_df = pd.read_csv("Data_3173_TS/hr_alarm_low.csv")  
extract_24(hr_alarm_low_df, master_neonates, 'Data_3145_TS_24/hr_alarm_low_24.csv')

## 1.6 SaO2 Alarm High 

In [91]:
sao2_alarm_high_df = pd.read_csv("Data_3173_TS/sao2_alarm_high.csv")  
extract_24(sao2_alarm_high_df, master_neonates, 'Data_3145_TS_24/sao2_alarm_high_24.csv')

## 1.7 SaO2 Alarm Low 

In [92]:
sao2_alarm_low_df = pd.read_csv("Data_3173_TS/sao2_alarm_low.csv")  
extract_24(sao2_alarm_low_df, master_neonates, 'Data_3145_TS_24/sao2_alarm_low_24.csv')

## 1.8 BP Cuff Diastolic 

In [93]:
bp_cuff_diastolic_df = pd.read_csv("Data_3173_TS/bp_cuff_diastolic.csv")  
extract_24(bp_cuff_diastolic_df, master_neonates, 'Data_3145_TS_24/bp_cuff_diastolic_24.csv')

## 1.9 BP Cuff Mean 

In [94]:
bp_cuff_mean_df = pd.read_csv("Data_3173_TS/bp_cuff_mean.csv")  
extract_24(bp_cuff_mean_df, master_neonates, 'Data_3145_TS_24/bp_cuff_mean_24.csv')

## 1.10 BP Cuff Systolic 

In [95]:
bp_cuff_systolic_df = pd.read_csv("Data_3173_TS/bp_cuff_systolic.csv")  
extract_24(bp_cuff_systolic_df, master_neonates, 'Data_3145_TS_24/bp_cuff_systolic_24.csv')

## 1.11 Temperature 

In [96]:
temperature_df = pd.read_csv("Data_3173_TS/temperature.csv")  
extract_24(temperature_df, master_neonates, 'Data_3145_TS_24/temperature_24.csv')

## 1.12 Skin Temperature 

In [97]:
skin_temperature_df = pd.read_csv("Data_3173_TS/skin_temperature.csv")  
extract_24(skin_temperature_df, master_neonates, 'Data_3145_TS_24/skin_temperature_24.csv')

## 1.13 Glucometer

In [98]:
glucometer_df = pd.read_csv("Data_3173_TS/glucometer.csv")  
extract_24(glucometer_df, master_neonates, 'Data_3145_TS_24/glucometer_24.csv')

## Other Chart Event Features 

In [105]:
def extract_24_bw(df, master_ids, filename):
    ## Renaming columns 
    df.rename(columns = {'subject_id_x': 'subject_id'}, inplace=True) 
    
    ## Dropping extra columns 
    df.drop(['subject_id_y', 'DISCHTIME', 'Dead'], axis=1, inplace=True)
    
    ## Merging with master neonates 
    df = pd.merge(df, master_ids, on='HADM_ID', how='inner')
    
    ## Saving into a given file 
    df.to_csv(filename, index=False)

## 2.1 Birth Weight 

In [99]:
birth_weight_df = pd.read_csv("Data_3173_SV/birth_weight.csv") 

In [100]:
birth_weight_df.head(2)

,Unnamed: 0,subject_id_x,HADM_ID,CHARTTIME,VALUENUM,subject_id_y,ADMITTIME,DISCHTIME,Dead
0,1,10,184167,2103-06-28 10:00:00,1.385,10,2103-06-28 11:36:00,2103-07-06 12:10:00,0
1,2,39,106266,2114-11-29 22:00:00,2.775,39,2114-11-29 21:04:00,2114-12-09 15:10:00,0


In [102]:
extract_24_bw(birth_weight_df, master_neonates,'Data_3145_SV_24/birth_weight_24.csv')

## 2.2 Head Circ 

In [103]:
head_circ_df = pd.read_csv("Data_3173_SV/head_circ.csv") 

In [104]:
head_circ_df.head(2)

,HADM_ID,CHARTTIME,VALUENUM,subject_id_x,subject_id_y,ADMITTIME,DISCHTIME,Dead
0,184167,2103-06-28 10:00:00,27.5,10,10,2103-06-28 11:36:00,2103-07-06 12:10:00,0
1,106266,2114-11-29 22:00:00,34.0,39,39,2114-11-29 21:04:00,2114-12-09 15:10:00,0


In [106]:
extract_24_bw(head_circ_df, master_neonates,'Data_3145_SV_24/head_circ_24.csv')

## Blood Test 

## 3.1 Bands 

In [107]:
bands_df = pd.read_csv("Data_3173_SV/bands.csv") 

In [108]:
bands_df.head(2)

,subject_id_x,HADM_ID,CHARTTIME,VALUENUM,subject_id_y,ADMITTIME,DISCHTIME,Dead
0,10824,143406,2126-06-24 12:05:00,6.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0
1,10824,143406,2126-06-23 23:05:00,3.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0


In [109]:
extract_24(bands_df, master_neonates,'Data_3145_SV_24/bands_24.csv')

## 3.2 Eosinophils

In [110]:
eosinophils_df = pd.read_csv("Data_3173_SV/eosinophils.csv")  
eosinophils_df.head(2)

,subject_id_x,HADM_ID,CHARTTIME,VALUENUM,subject_id_y,ADMITTIME,DISCHTIME,Dead
0,10824,143406,2126-06-24 12:05:00,1.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0
1,10824,143406,2126-06-23 23:05:00,5.0,10824,2126-06-12 22:08:00,2126-07-15 13:40:00,0


In [111]:
extract_24(eosinophils_df, master_neonates,'Data_3145_SV_24/eosinophils_24.csv')

## 3.3 Lymphs

In [112]:
lymphs_df = pd.read_csv("Data_3173_SV/lymphs.csv")  
extract_24(lymphs_df, master_neonates,'Data_3145_SV_24/lymphs_24.csv')

## 3.4 MONOs

In [113]:
monos_df = pd.read_csv("Data_3173_SV/monos.csv")  
extract_24(monos_df, master_neonates,'Data_3145_SV_24/monos_24.csv')

## 3.5 NEUTs

In [114]:
neuts_df = pd.read_csv("Data_3173_SV/neuts.csv")  
extract_24(neuts_df, master_neonates,'Data_3145_SV_24/neuts_24.csv')

## 3.6 Platelet

In [115]:
platelet_df = pd.read_csv("Data_3173_SV/platelet.csv")  
extract_24(platelet_df, master_neonates,'Data_3145_SV_24/platelet_24.csv')

## Statistical Data 

In [137]:
master_neonates_ids = pd.read_csv("neonates_admissions.csv")

In [138]:
master_neonates_ids = pd.merge(master_neonates_ids, master_neonates, on='HADM_ID', how='inner')

In [139]:
master_neonates_ids.columns 

Index(['subject_id', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'ETHNICITY',
       'GENDER', 'EXPIRE_FLAG'],
      dtype='object')

In [140]:
master_neonates_ids.drop(['subject_id', 'DISCHTIME', 'ETHNICITY',
       'GENDER', 'EXPIRE_FLAG'], axis=1, inplace=True)

In [141]:
master_neonates_ids.head(2)

,HADM_ID,ADMITTIME
0,106266,2114-11-29 21:04:00
1,110872,2168-10-24 23:48:00


In [142]:
master_neonates_ids.shape 

(3145, 2)

## 4.1 DateTimeEvents 

In [116]:
datetimeevents_df = pd.read_csv("Data_3173_Stat/datetimeevents.csv")

In [117]:
datetimeevents_df.head()

,HADM_ID,subject_id,CHARTTIME
0,106266,39,2114-11-29 22:00:00
1,106266,39,2114-11-29 23:00:00
2,106266,39,2114-11-30 00:00:00
3,106266,39,2114-11-30 01:00:00
4,106266,39,2114-11-30 02:00:00


In [143]:
datetimeevents_df = pd.merge(datetimeevents_df, master_neonates_ids, on='HADM_ID', how='inner')

In [144]:
datetimeevents_df.head(2)

,HADM_ID,subject_id,CHARTTIME,ADMITTIME
0,106266,39,2114-11-29 22:00:00,2114-11-29 21:04:00
1,106266,39,2114-11-29 23:00:00,2114-11-29 21:04:00


In [147]:
datetimeevents_df.shape 

(180007, 5)

In [145]:
datetimeevents_df['CHARTTIME'] = datetimeevents_df['CHARTTIME'].astype('datetime64[ns]')
datetimeevents_df['ADMITTIME'] = datetimeevents_df['ADMITTIME'].astype('datetime64[ns]') 
    
## Calculating time interval 
datetimeevents_df['TIME_INTERVAL'] = datetimeevents_df['CHARTTIME'] - datetimeevents_df['ADMITTIME']

## Extracting only 24 hours data 
datetimeevents_24 = datetimeevents_df[(datetimeevents_df.TIME_INTERVAL <= '1 days') & (datetimeevents_df.TIME_INTERVAL >= '-1 days')]

datetimeevents_24.drop(['TIME_INTERVAL'], axis=1, inplace=True) 
    
## Saving into a given file 
datetimeevents_24.to_csv("Data_3145_Stat_24/datetimeevents_24.csv", index=False)

In [146]:
datetimeevents_24.shape

(19182, 4)

## 4.2 InputEvents 

In [148]:
inputevents_df = pd.read_csv("Data_3173_Stat/neonates_inputevents.csv")

In [149]:
inputevents_df.head(2)

,HADM_ID,subject_id,CHARTTIME,ITEMID,AMOUNT,LABEL
0,106266,39,2114-11-29 21:30:00,30187,NaN,D10W
1,106266,39,2114-11-29 22:00:00,30187,2.9,D10W


In [152]:
inputevents_df.shape 

(2028792, 8)

In [150]:
inputevents_df = pd.merge(inputevents_df, master_neonates_ids, on='HADM_ID', how='inner')

inputevents_df['CHARTTIME'] = inputevents_df['CHARTTIME'].astype('datetime64[ns]')
inputevents_df['ADMITTIME'] = inputevents_df['ADMITTIME'].astype('datetime64[ns]') 
    
## Calculating time interval 
inputevents_df['TIME_INTERVAL'] = inputevents_df['CHARTTIME'] - inputevents_df['ADMITTIME']

## Extracting only 24 hours data 
inputevents_24 = inputevents_df[(inputevents_df.TIME_INTERVAL <= '1 days') & (inputevents_df.TIME_INTERVAL >= '-1 days')]

inputevents_24.drop(['TIME_INTERVAL'], axis=1, inplace=True) 
    
## Saving into a given file 
inputevents_24.to_csv("Data_3145_Stat_24/inputevents_24.csv", index=False)

In [151]:
inputevents_24.shape 

(115545, 7)

In [153]:
inputevents_24['HADM_ID'].nunique()

3091

## 4.3 Microbiologyevents

In [154]:
microbiologyevents_df = pd.read_csv("Data_3173_Stat/neonates_microbiologyevents.csv")
microbiologyevents_df.head(2)

,HADM_ID,subject_id,CHARTTIME,SPEC_ITEMID,ORG_ITEMID,AB_ITEMID
0,106266,39,2114-11-29 21:45:00,70014.0,0.0,0.0
1,104518,66,2188-08-25 00:45:00,70014.0,0.0,0.0


In [155]:
microbiologyevents_df.shape 

(10352, 6)

In [156]:
microbiologyevents_df = pd.merge(microbiologyevents_df, master_neonates_ids, on='HADM_ID', how='inner')

microbiologyevents_df['CHARTTIME'] = microbiologyevents_df['CHARTTIME'].astype('datetime64[ns]')
microbiologyevents_df['ADMITTIME'] = microbiologyevents_df['ADMITTIME'].astype('datetime64[ns]') 
    
## Calculating time interval 
microbiologyevents_df['TIME_INTERVAL'] = microbiologyevents_df['CHARTTIME'] - microbiologyevents_df['ADMITTIME']

## Extracting only 24 hours data 
microbiologyevents_24 = microbiologyevents_df[(microbiologyevents_df.TIME_INTERVAL <= '1 days') & (microbiologyevents_df.TIME_INTERVAL >= '-1 days')]

microbiologyevents_24.drop(['TIME_INTERVAL'], axis=1, inplace=True) 
    
## Saving into a given file 
microbiologyevents_24.to_csv("Data_3145_Stat_24/microbiologyevents_24.csv", index=False)

In [157]:
microbiologyevents_24.shape

(3283, 7)

## 4.4 Outputevents

In [158]:
outputevents_df = pd.read_csv("Data_3173_Stat/neonates_outputevents.csv")
outputevents_df.head(2)

,HADM_ID,subject_id,CHARTTIME,ITEMID,VALUE,CGID,LABEL
0,106266,39,2114-11-30 05:00:00,43175,9.0,15658,Urine .
1,106266,39,2114-11-30 13:00:00,43175,33.0,14440,Urine .


In [161]:
outputevents_df.shape 

(113199, 9)

In [159]:
outputevents_df = pd.merge(outputevents_df, master_neonates_ids, on='HADM_ID', how='inner')

outputevents_df['CHARTTIME'] = outputevents_df['CHARTTIME'].astype('datetime64[ns]')
outputevents_df['ADMITTIME'] = outputevents_df['ADMITTIME'].astype('datetime64[ns]') 
    
## Calculating time interval 
outputevents_df['TIME_INTERVAL'] = outputevents_df['CHARTTIME'] - outputevents_df['ADMITTIME']

## Extracting only 24 hours data 
outputevents_24 = outputevents_df[(outputevents_df.TIME_INTERVAL <= '1 days') & (outputevents_df.TIME_INTERVAL >= '-1 days')]

outputevents_24.drop(['TIME_INTERVAL'], axis=1, inplace=True) 
    
## Saving into a given file 
outputevents_24.to_csv("Data_3145_Stat_24/outputevents_24.csv", index=False)

In [160]:
outputevents_24.shape 

(14320, 8)

## 4.5 Prescriptions

In [162]:
prescriptions_df = pd.read_csv("Data_3173_Stat/neonates_prescriptions.csv")
prescriptions_df.head(2)

,HADM_ID,subject_id,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DOSE_VAL_RX
0,104518,66,2188-08-25 00:00:00,2188-08-27 00:00:00,BASE,Syringe (Neonatal) *D5W*,2.4
1,104518,66,2188-08-25 00:00:00,2188-08-27 00:00:00,MAIN,NEO*IV*AMPicillin Sodium,470


In [163]:
prescriptions_df = pd.merge(prescriptions_df, master_neonates_ids, on='HADM_ID', how='inner')

prescriptions_df['STARTDATE'] = prescriptions_df['STARTDATE'].astype('datetime64[ns]')
prescriptions_df['ADMITTIME'] = prescriptions_df['ADMITTIME'].astype('datetime64[ns]') 
    
## Calculating time interval 
prescriptions_df['TIME_INTERVAL'] = prescriptions_df['STARTDATE'] - prescriptions_df['ADMITTIME']

## Extracting only 24 hours data 
prescriptions_24 = prescriptions_df[(prescriptions_df.TIME_INTERVAL <= '1 days') & (prescriptions_df.TIME_INTERVAL >= '-1 days')]

prescriptions_24.drop(['TIME_INTERVAL'], axis=1, inplace=True) 
    
## Saving into a given file 
prescriptions_24.to_csv("Data_3145_Stat_24/prescriptions_24.csv", index=False)